# 05-3 Gemma

<table align="left"><tr><td>
<a href="https://colab.research.google.com/github/rickiepark/hm-dl/blob/main/05-3-gemma-ko.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="코랩에서 실행하기"/></a>
</td></tr></table>

코랩에서 이 노트북을 실행하려면 A100 또는 High-RAM CPU 런타임을 사용해야 합니다.

In [1]:
import keras
from keras import layers
import keras_nlp

In [2]:
def make_causal_mask(seq_len):
    n_hori = keras.ops.arange(seq_len)
    n_vert = keras.ops.expand_dims(n_hori, axis=-1)
    mask = n_vert >= n_hori
    return mask

In [3]:
def make_attention_mask(padding_mask):
    # padding_mask 크기가 (2, 5)라고 가정해 보죠.
    batch_size, seq_len = keras.ops.shape(padding_mask)
    # causal_mask 크기는 (5, 5)가 됩니다.
    causal_mask = make_causal_mask(seq_len)
    # 배치 차원을 추가해 (2, 5, 5)로 만듭니다.
    causal_mask = keras.ops.broadcast_to(causal_mask, (batch_size, seq_len, seq_len))
    # 브로드캐스팅을 위해 padding_mask 크기를 (2, 1, 5)로 만듭니다.
    padding_mask = keras.ops.expand_dims(padding_mask, axis=1)
    return keras.ops.minimum(causal_mask, padding_mask)

In [4]:
class AttentionMask(keras.Layer):
    def call(self, padding_mask):
        return make_attention_mask(padding_mask)

## 젬마 모델 사용하기

In [12]:
from transformers import pipeline, set_seed

In [13]:
gemma_pipe = pipeline("text-generation", model="beomi/gemma-ko-2b")
set_seed(42)
gemma_pipe('봄이 오면', max_length=20, truncation=True)

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/555 [00:00<?, ?B/s]

Device set to use cuda:0


OutOfMemoryError: CUDA out of memory. Tried to allocate 128.00 MiB. GPU 0 has a total capacity of 39.56 GiB of which 84.88 MiB is free. Process 9227 has 39.46 GiB memory in use. Of the allocated memory 7.04 GiB is allocated by PyTorch, and 1.81 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

## 젬마 2

In [ ]:
def gemma2_decoder(x, padding_mask, num_query_heads, num_key_value_heads,
                  interm_dim, hidden_dim, head_dim):
    # 어텐션 마스크를 계산합니다.
    attention_mask = AttentionMask()(padding_mask)
    # 스킵 연결을 준비합니다.
    residual = x
    x = RMSNormalization()(x)
    # 멀티 헤드 어텐션을 통과합니다.
    gemma_attention = CachedGemmaAttention(head_dim=head_dim,
                                           num_query_heads=num_query_heads,
                                           num_key_value_heads=num_key_value_heads,
                                           use_sliding_window_attention=True,
                                           dropout=0.0)
    x = gemma_attention(x, attention_mask)
    # 포스트 정규화
    x = RMSNormalization()(x)
    # 스킵 연결
    x = x + residual
    # 스킵 연결을 준비합니다.
    residual = x
    # 위치별 피드 포워드 네트워크
    x = RMSNormalization()(x)
    x1 = layers.Dense(interm_dim, activation='gelu', use_bias=False)(x)
    x2 = layers.Dense(interm_dim, use_bias=False)(x)
    x = x1 * x2
    x = layers.Dense(hidden_dim, use_bias=False)(x)
    # 포스트 정규화
    x = RMSNormalization()(x)
    # 스킵 연결
    x = x + residual
    return x

In [ ]:
# Gemma2 2B
vocab_size = 256000
num_layers = 26
num_query_heads = 8
num_key_value_heads = 4
interm_dim = 9216
hidden_dim = 2304
head_dim = 256

token_ids = keras.Input(shape=(None,))
padding_mask = keras.Input(shape=(None,))

token_embedding_layer = ReversibleEmbedding(vocab_size, hidden_dim)
x = token_embedding_layer(token_ids)
x = layers.Lambda(lambda x: x * keras.ops.sqrt(hidden_dim))(x)

for _ in range(num_layers):
    x = gemma2_decoder(x, padding_mask, num_query_heads, num_key_value_heads,
                      interm_dim, hidden_dim, head_dim)

x = RMSNormalization()(x)
outputs = token_embedding_layer(x, reverse=True)
model = keras.Model(inputs=(token_ids, padding_mask),
                    outputs=(outputs))
model.summary(line_length=100)

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                ┃ Output Shape            ┃        Param # ┃ Connected to            ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)  │ (None, None)            │              0 │ -                       │
├─────────────────────────────┼─────────────────────────┼────────────────┼─────────────────────────┤
│ reversible_embedding_1      │ (None, None, 256000)    │    589,824,000 │ input_layer_2[0][0],    │
│ (ReversibleEmbedding)       │                         │                │ rms_normalization_141[… │
├─────────────────────────────┼─────────────────────────┼────────────────┼─────────────────────────┤
│ lambda_1 (Lambda)           │ (None, None, 2304)      │              0 │ reversible_embedding_1… │
├─────────────────────────────┼─────────────────────────┼────────────────┼─────────────────────────┤
│ input_layer_3 (InputLayer)  │ (None, None)            │              0 │ -                       │
├─────────────────────────────┼─────────────────────────┼────────────────┼─────────────────────────┤
│ rms_normalization_37        │ (None, None, 2304)      │          2,304 │ lambda_1[0][0]          │
│ (RMSNormalization)          │                         │                │                         │
├─────────────────────────────┼─────────────────────────┼────────────────┼─────────────────────────┤
│ attention_mask_18           │ (None, None, None)      │              0 │ input_layer_3[0][0]     │
│ (AttentionMask)             │                         │                │                         │
├─────────────────────────────┼─────────────────────────┼────────────────┼─────────────────────────┤
│ cached_gemma_attention_18   │ (None, None, 2304)      │     14,155,776 │ rms_normalization_37[0… │
│ (CachedGemmaAttention)      │                         │                │ attention_mask_18[0][0] │
├─────────────────────────────┼─────────────────────────┼────────────────┼─────────────────────────┤
│ rms_normalization_38        │ (None, None, 2304)      │          2,304 │ cached_gemma_attention… │
│ (RMSNormalization)          │                         │                │                         │
├─────────────────────────────┼─────────────────────────┼────────────────┼─────────────────────────┤
│ add_36 (Add)                │ (None, None, 2304)      │              0 │ rms_normalization_38[0… │
│                             │                         │                │ lambda_1[0][0]          │
├─────────────────────────────┼─────────────────────────┼────────────────┼─────────────────────────┤
│ rms_normalization_39        │ (None, None, 2304)      │          2,304 │ add_36[0][0]            │
│ (RMSNormalization)          │                         │                │                         │
├─────────────────────────────┼─────────────────────────┼────────────────┼─────────────────────────┤
│ dense_54 (Dense)            │ (None, None, 9216)      │     21,233,664 │ rms_normalization_39[0… │
├─────────────────────────────┼─────────────────────────┼────────────────┼─────────────────────────┤
│ dense_55 (Dense)            │ (None, None, 9216)      │     21,233,664 │ rms_normalization_39[0… │
├─────────────────────────────┼─────────────────────────┼────────────────┼─────────────────────────┤
│ multiply_19 (Multiply)      │ (None, None, 9216)      │              0 │ dense_54[0][0],         │
│                             │                         │                │ dense_55[0][0]          │
├─────────────────────────────┼─────────────────────────┼────────────────┼─────────────────────────┤
│ dense_56 (Dense)            │ (None, None, 2304)      │     21,233,664 │ multiply_19[0][0]       │
├─────────────────────────────┼─────────────────────────┼────────────────┼─────────────────────────┤
│ rms_normalization_40        │ (None, None, 2304)      │    

 Total params: 2,614,341,888 (9.74 GB)

 Trainable params: 2,614,341,888 (9.74 GB)

 Non-trainable params: 0 (0.00 B)